
## Применить алгоритмы ARL : Apriori, ECLAT, FP-Growth к данным

### Использованы данные из работы №12

В работе № 12 была поставлена цель достич максимально возможных цифр roc_auc. 
В этой работе попробуем улучшить этот результат.
Преобразования исходного дататасета удалены - взят преобразованый и готовые параметры модели из прошлой работы.

###### Описание набора данных
Данные представляют собой исторические данные по 251503 заемщикам (https://www.kaggle.com/c/GiveMeSomeCredit/data ). Задача является бинарной классификацией. 


In [34]:
import numpy as np, pandas as pd

import pyfpgrowth
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import xgboost as xgb

from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics.cluster import v_measure_score

import warnings
warnings.simplefilter('ignore')


Преобразованные ранее данные.

In [4]:
pd.read_csv('df_Depend.csv', index_col=0).head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,Depend_NaN_1,Depend_mix
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,0.0,4.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0.0,0.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,0.0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0,0.0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0,0.0,0.0


In [5]:
df_Depend1 = pd.read_csv('df_Depend.csv', index_col=0)
df_Depend1.shape

(150000, 13)

Создание двух наболров данных - для применения алгоритмов и контрольные.
Для применения алгоритма используется часть данных.
Основная цель работы научиться применять алгоритмы ARL и посмотреть их эффективность.

In [41]:
df_bin = df_Depend1.copy()


# случайный отбор 70% записей с перемешиванием
train_bin = df_bin.sample(frac=0.7, random_state=100) 
# удаление из df_bin всех train_bin
test_bin = df_bin.drop(train_bin.index) 

train_dec = train_bin[['SeriousDlqin2yrs']].copy()
test_dec = test_bin[['SeriousDlqin2yrs']].copy()

# биннинг переменных на основе децилей 

train_dec['age_dc'] = pd.qcut(train_bin['age'], 8)
train_dec['MI_dc'] = pd.qcut(train_bin['MonthlyIncome'], 8)
train_dec['NOD_dc'] = pd.qcut(train_bin['NumberOfDependents'], 4, duplicates='drop')
train_dec['NREOL_dc'] = pd.qcut(train_bin['NumberRealEstateLoansOrLines'], 4, duplicates='drop')

test_dec['age_dc'] = pd.qcut(test_bin['age'], 8)
test_dec['MI_dc'] = pd.qcut(test_bin['MonthlyIncome'], 8)
test_dec['NOD_dc'] = pd.qcut(test_bin['NumberOfDependents'], 4, duplicates='drop')
test_dec['NREOL_dc'] = pd.qcut(test_bin['NumberRealEstateLoansOrLines'], 4, duplicates='drop')

train_dec.head()

,SeriousDlqin2yrs,age_dc,MI_dc,NOD_dc,NREOL_dc
149312,0,"(20.999, 34.0]","(3404.0, 4500.0]","(-0.001, 1.0]","(-0.001, 1.0]"
19950,0,"(57.0, 63.0]","(5778.0, 7416.0]","(-0.001, 1.0]","(-0.001, 1.0]"
93749,0,"(57.0, 63.0]","(7416.0, 10000.0]","(-0.001, 1.0]","(-0.001, 1.0]"
129753,0,"(52.0, 57.0]","(-0.001, 1102.0]","(-0.001, 1.0]","(-0.001, 1.0]"
71411,0,"(57.0, 63.0]","(2383.75, 3404.0]","(-0.001, 1.0]","(1.0, 2.0]"


In [42]:
# контрольный набор данных без децилей
train_0 = train_bin.copy()
test_0 = test_bin.copy()

In [43]:
# дамми
train_dec = pd.get_dummies(train_dec, prefix_sep='_', drop_first=True)
test_dec = pd.get_dummies(test_dec, prefix_sep='_', drop_first=True)
print(train_dec.shape)
train_dec.head(3)

(105000, 18)


,SeriousDlqin2yrs,"age_dc_(34.0, 41.0]","age_dc_(41.0, 47.0]","age_dc_(47.0, 52.0]","age_dc_(52.0, 57.0]","age_dc_(57.0, 63.0]","age_dc_(63.0, 70.0]","age_dc_(70.0, 109.0]","MI_dc_(1102.0, 2383.75]","MI_dc_(2383.75, 3404.0]","MI_dc_(3404.0, 4500.0]","MI_dc_(4500.0, 5778.0]","MI_dc_(5778.0, 7416.0]","MI_dc_(7416.0, 10000.0]","MI_dc_(10000.0, 3008750.0]","NOD_dc_(1.0, 13.0]","NREOL_dc_(1.0, 2.0]","NREOL_dc_(2.0, 54.0]"
149312,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
19950,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
93749,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


In [44]:

y_train_dec = train_dec['SeriousDlqin2yrs']
X_train_dec = train_dec.drop(['SeriousDlqin2yrs'],axis=1)#,

y_test_dec = test_dec['SeriousDlqin2yrs']
X_test_dec = test_dec.drop(['SeriousDlqin2yrs'],axis=1)
X_train_dec.shape

(105000, 17)

Применение алгоритма apriori из frequent



Спасибо группе!!!

In [45]:
fr_itemsets = apriori(X_train_dec, min_support=0.03, use_colnames=True)
fr_itemsets.head()


,support,itemsets
0,0.126048,"(age_dc_(34.0, 41.0])"
1,0.137238,"(age_dc_(41.0, 47.0])"
2,0.122781,"(age_dc_(47.0, 52.0])"
3,0.117352,"(age_dc_(52.0, 57.0])"
4,0.137924,"(age_dc_(57.0, 63.0])"


In [46]:
# сортировка по убыванию 'support'
fr_itemsets.sort_values('support', ascending=False)

,support,itemsets
14,0.219010,"(NOD_dc_(1.0, 13.0])"
15,0.209800,"(NREOL_dc_(1.0, 2.0])"
4,0.137924,"(age_dc_(57.0, 63.0])"
1,0.137238,"(age_dc_(41.0, 47.0])"
9,0.129752,"(MI_dc_(3404.0, 4500.0])"
12,0.126990,"(MI_dc_(7416.0, 10000.0])"
0,0.126048,"(age_dc_(34.0, 41.0])"
11,0.125190,"(MI_dc_(5778.0, 7416.0])"
8,0.125067,"(MI_dc_(2383.75, 3404.0])"
7,0.124981,"(MI_dc_(1102.0, 2383.75])"


In [47]:
%%time
# то же через fpgrowth 
fpg_train = fpgrowth(X_train_dec, min_support=0.030, use_colnames=True)
fpg_train.sort_values('support', ascending=False)

Wall time: 2.23 s


,support,itemsets
11,0.219010,"(NOD_dc_(1.0, 13.0])"
5,0.209800,"(NREOL_dc_(1.0, 2.0])"
1,0.137924,"(age_dc_(57.0, 63.0])"
16,0.137238,"(age_dc_(41.0, 47.0])"
0,0.129752,"(MI_dc_(3404.0, 4500.0])"
3,0.126990,"(MI_dc_(7416.0, 10000.0])"
12,0.126048,"(age_dc_(34.0, 41.0])"
2,0.125190,"(MI_dc_(5778.0, 7416.0])"
6,0.125067,"(MI_dc_(2383.75, 3404.0])"
14,0.124981,"(MI_dc_(1102.0, 2383.75])"


Видно, что алгоритмы отрабатываю одинаково.

Для X_test_dec

In [48]:
fpg_test = fpgrowth(X_test_dec, min_support=0.030, use_colnames=True)

fpg_test.sort_values('support', ascending=False)

,support,itemsets
4,0.219156,"(NOD_dc_(1.0, 20.0])"
7,0.210956,"(NREOL_dc_(1.0, 2.0])"
12,0.140178,"(age_dc_(57.0, 63.0])"
11,0.135756,"(age_dc_(41.0, 47.0])"
8,0.130689,"(MI_dc_(5800.0, 7500.0])"
0,0.127644,"(age_dc_(34.0, 41.0])"
2,0.127556,"(age_dc_(47.0, 52.0])"
13,0.126267,"(MI_dc_(3458.0, 4500.0])"
1,0.125044,"(MI_dc_(2401.0, 3458.0])"
15,0.125000,"(MI_dc_(1132.0, 2401.0])"


Создание новых переменных и их коррекция для train и test про результатам применения алгорима

In [53]:

train_bin['NREOL'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['NREOL_dc_(1.0, 2.0]']
train_bin['MI'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['MI_dc_(10000.0, 3008750.0]']
train_bin['age_41.0'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['age_dc_(41.0, 47.0]']
train_bin['age_34.0'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['age_dc_(34.0, 41.0]']
train_bin['age_47.0'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['age_dc_(47.0, 52.0]']

train_bin['NREOL_MI'] = train_dec['NREOL_dc_(1.0, 2.0]'] & train_dec['MI_dc_(10000.0, 3008750.0]']
train_bin['NOD_MI'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['MI_dc_(7416.0, 10000.0]']
train_bin['MI_NREOL'] = train_dec['MI_dc_(7416.0, 10000.0]'] & train_dec['NREOL_dc_(1.0, 2.0]']
train_bin['NREOL_age'] = train_dec['NREOL_dc_(1.0, 2.0]'] & train_dec['age_dc_(41.0, 47.0]']
train_bin['NOD_MI_1'] = train_dec['NOD_dc_(1.0, 13.0]'] & train_dec['MI_dc_(5778.0, 7416.0]']
train_bin['NREOL_age_2'] = train_dec['NREOL_dc_(1.0, 2.0]'] & train_dec['age_dc_(57.0, 63.0]']
train_bin['MI_NREOL_2'] = train_dec['MI_dc_(5778.0, 7416.0]'] & train_dec['NREOL_dc_(1.0, 2.0]']


train_bin['NOD_dc_1.0_13.0'] = train_dec['NOD_dc_(1.0, 13.0]']
train_bin['NREOL_dc_1.0_2.0'] = train_dec['NREOL_dc_(1.0, 2.0]']
train_bin['age_dc_57.0_63.0'] = train_dec['age_dc_(57.0, 63.0]']
train_bin['age_dc_41.0_47.0'] = train_dec['age_dc_(41.0, 47.0]']
train_bin['MI_dc_3404.0_4500.0'] = train_dec['MI_dc_(3404.0, 4500.0]']
train_bin['MI_dc_7416.0_10000.0'] = train_dec['MI_dc_(7416.0, 10000.0]']

train_bin['age_dc_34_41'] = train_dec['age_dc_(34.0, 41.0]']
train_bin['MI_dc_5778_416'] = train_dec['MI_dc_(5778.0, 7416.0]']
train_bin['MI_dc_2383_3404'] = train_dec['MI_dc_(2383.75, 3404.0]']
train_bin['MI_dc_1102_2383'] = train_dec['MI_dc_(1102.0, 2383.75]']
train_bin['MI_dc_10000_3008750'] = train_dec['MI_dc_(10000.0, 3008750.0]']
train_bin['age_dc_47_52'] = train_dec['age_dc_(47.0, 52.0]']



test_bin['NREOL'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['NREOL_dc_(1.0, 2.0]']
test_bin['MI'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['MI_dc_(10000.0, 1794060.0]']
test_bin['age_41.0'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['age_dc_(41.0, 47.0]']
test_bin['age_34.0'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['age_dc_(34.0, 41.0]']
test_bin['age_47.0'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['age_dc_(47.0, 52.0]']

test_bin['NREOL_MI'] = test_dec['NREOL_dc_(1.0, 2.0]'] & test_dec['MI_dc_(10000.0, 1794060.0]']
test_bin['NOD_MI'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['MI_dc_(7500.0, 10000.0]']
test_bin['MI_NREOL'] = test_dec['MI_dc_(7500.0, 10000.0]'] & test_dec['NREOL_dc_(1.0, 2.0]']
test_bin['NREOL_age'] = test_dec['NREOL_dc_(1.0, 2.0]'] & test_dec['age_dc_(41.0, 47.0]']
test_bin['NOD_MI_1'] = test_dec['NOD_dc_(1.0, 20.0]'] & test_dec['MI_dc_(5800.0, 7500.0]']
test_bin['NREOL_age_2'] = test_dec['NREOL_dc_(1.0, 2.0]'] & test_dec['age_dc_(57.0, 63.0]']
test_bin['MI_NREOL_2'] = test_dec['MI_dc_(5800.0, 7500.0]'] & test_dec['NREOL_dc_(1.0, 2.0]']


test_bin['NOD_dc_1.0_13.0'] = test_dec['NOD_dc_(1.0, 20.0]']
test_bin['NREOL_dc_1.0_2.0'] = test_dec['NREOL_dc_(1.0, 2.0]']
test_bin['age_dc_57.0_63.0'] = test_dec['age_dc_(57.0, 63.0]']
test_bin['age_dc_41.0_47.0'] = test_dec['age_dc_(41.0, 47.0]']
test_bin['MI_dc_3404.0_4500.0'] = test_dec['MI_dc_(3458.0, 4500.0]']
test_bin['MI_dc_7416.0_10000.0'] = test_dec['MI_dc_(7500.0, 10000.0]']

test_bin['age_dc_34_41'] = test_dec['age_dc_(34.0, 41.0]']
test_bin['MI_dc_5778_416'] = test_dec['MI_dc_(5800.0, 7500.0]']
test_bin['MI_dc_2383_3404'] = test_dec['MI_dc_(2401.0, 3458.0]']
test_bin['MI_dc_1102_2383'] = test_dec['MI_dc_(1132.0, 2401.0]']
test_bin['MI_dc_10000_3008750'] = test_dec['MI_dc_(10000.0, 1794060.0]']
test_bin['age_dc_47_52'] = test_dec['age_dc_(47.0, 52.0]']

print(test_bin.shape)
test_bin.head(3)

(45000, 37)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,...,MI_dc_1102_2383,MI_dc_10000_3008750,age_dc_47_52,NREOL_MI,NOD_MI,MI_NREOL,NREOL_age,NOD_MI_1,NREOL_age_2,MI_NREOL_2
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,...,0,1,1,0,0,0,0,0,0,0
10,0,0.189169,57,0,0.606291,23684.0,9,0,4,0,...,0,1,0,0,0,0,0,0,0,0


In [54]:
y_train_bin = train_bin['SeriousDlqin2yrs']

X_train_bin = train_bin.drop(['SeriousDlqin2yrs',
#                               'DebtRatio',
#                               'RevolvingUtilizationOfUnsecuredLines',
#                               'NumberOfOpenCreditLinesAndLoans',
#                               'NumberOfTimes90DaysLate',
#                               'NumberRealEstateLoansOrLines',
#                               'NumberOfTime60-89DaysPastDueNotWorse',
#                               'NumberOfDependents',                            
#                               'MonthlyIncome', 
#                               'age'
                             ],axis=1)



y_test_bin = test_bin['SeriousDlqin2yrs']

X_test_bin = test_bin.drop(['SeriousDlqin2yrs',
#                             'DebtRatio',
#                             'RevolvingUtilizationOfUnsecuredLines',
#                             'NumberOfOpenCreditLinesAndLoans',
#                             'NumberOfTimes90DaysLate',
#                             'NumberRealEstateLoansOrLines',
#                             'NumberOfTime60-89DaysPastDueNotWorse',
#                             'NumberOfDependents',                            
#                             'MonthlyIncome', 
#                             'age'
                           ],axis=1)

X_train_bin.shape

(105000, 36)

Модель XGBClassifier с подобраными ранее параметрами

In [55]:

# model = xgb.XGBClassifier()
model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.95, gamma=0.85,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=None, n_estimators=589, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1, eta= 0.2 
              )

rfr = model.fit(X_train_bin, y_train_bin)

expected = y_test_bin
predicted = model.predict(X_test_bin)
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
print(metrics.classification_report(y_train_bin, model.predict(X_train_bin)))
print(metrics.confusion_matrix(y_train_bin, model.predict(X_train_bin)))
print(round((rfr.score(X_train_bin, y_train_bin))*100)/100),
print(round((rfr.score(X_test_bin, y_test_bin))*100)/100)
print(int(roc_auc_score(y_train_bin, model.predict_proba(X_train_bin)[:,1])*10000)/10000,'= roc_auc_score  predict_proba_train' )
print(int(roc_auc_score(expected, model.predict_proba(X_test_bin)[:,1])*10000)/10000,'= roc_auc_score  predict_proba_test' )

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     41973
           1       0.62      0.20      0.31      3027

    accuracy                           0.94     45000
   macro avg       0.78      0.60      0.64     45000
weighted avg       0.92      0.94      0.92     45000

[[41585   388]
 [ 2407   620]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     98001
           1       0.69      0.22      0.34      6999

    accuracy                           0.94    105000
   macro avg       0.82      0.61      0.65    105000
weighted avg       0.93      0.94      0.93    105000

[[97299   702]
 [ 5438  1561]]
0.94
0.94
0.8857 = roc_auc_score  predict_proba_train
0.8617 = roc_auc_score  predict_proba_test


In [479]:
# Подбор параметров
# XGBClassifier
X = X_train_bin 
y = y_train_bin 
Xcv = X_test_bin
ycv = y_test_bin 

def objective(space):
    clf = xgb.XGBClassifier(n_estimators =1000,colsample_bytree=space['colsample_bytree'],
                           learning_rate = .3,
                            max_depth = int(space['max_depth']),
                            min_child_weight = space['min_child_weight'],
                            subsample = space['subsample'],
                           gamma = space['gamma'])
    eval_set  = [( X, y), ( Xcv, ycv)]
    clf.fit(X, y,
            eval_set=eval_set, eval_metric="rmse",
            early_stopping_rounds=10,verbose=False)
    pred = clf.predict_proba(Xcv)[:, 1]
    accuracy = roc_auc_score(ycv, pred)
    return {'loss': -accuracy, 'status': STATUS_OK }

space_1 ={'max_depth': hp.quniform("x_max_depth", 4, 16, 1),
        'min_child_weight': hp.quniform ('x_min_child', 1, 10, 1),
        'subsample': hp.uniform ('x_subsample', 0.7, 1),
        'gamma' : hp.uniform ('x_gamma', 0.1,0.5),
        'colsample_bytree' : hp.uniform ('x_colsample_bytree', 0.7,1),
        'reg_lambda' : hp.uniform ('x_reg_lambda', 0,1)
    }

space_2 = {
            'n_estimators': hp.choice('n_estimators', range(100, 1000)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
            'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            'nthread': 4,
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1
        }

space = space_1
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
print(best)

100%|██████████| 50/50 [16:21<00:00, 19.63s/it, best loss: -0.8622269136771274]
{'x_colsample_bytree': 0.7182364579625985, 'x_gamma': 0.1298161918642156, 'x_max_depth': 4.0, 'x_min_child': 2.0, 'x_reg_lambda': 0.8801752102900298, 'x_subsample': 0.8687114774746062}


### Обучение на контрольной группе, без применения ARL:

In [37]:

y_train_0 = train_0['SeriousDlqin2yrs']
X_train_0 = train_0.drop(['SeriousDlqin2yrs'],axis=1)

y_test_0 = test_0['SeriousDlqin2yrs']
X_test_0 = test_0.drop(['SeriousDlqin2yrs'],axis=1)
X_train_0.shape

(105000, 12)

In [39]:

from sklearn.metrics import accuracy_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import numpy as np
import xgboost as xgb
# model = xgb.XGBClassifier()
model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.95, gamma=0.85,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=None, n_estimators=589, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1, eta= 0.2 
              )


# %%time
# model = xgb.XGBClassifier()
rfr = model.fit(X_train_0, y_train_0)

# print(model)
# make predictions
expected = y_test_0
predicted = model.predict(X_test_0)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
print(metrics.classification_report(y_train_0, model.predict(X_train_0)))
print(metrics.confusion_matrix(y_train_0, model.predict(X_train_0)))
print(round((rfr.score(X_train_0, y_train_0))*100)/100),
print(round((rfr.score(X_test_0, y_test_0))*100)/100)
print(int(roc_auc_score(y_train_0, model.predict_proba(X_train_0)[:,1])*10000)/10000,'= roc_auc_score  predict_proba_train' )
print(int(roc_auc_score(expected, model.predict_proba(X_test_0)[:,1])*10000)/10000,'= roc_auc_score  predict_proba_test' )

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     41973
           1       0.61      0.21      0.31      3027

    accuracy                           0.94     45000
   macro avg       0.78      0.60      0.64     45000
weighted avg       0.92      0.94      0.92     45000

[[41578   395]
 [ 2404   623]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97     98001
           1       0.69      0.22      0.34      6999

    accuracy                           0.94    105000
   macro avg       0.82      0.61      0.65    105000
weighted avg       0.93      0.94      0.93    105000

[[97283   718]
 [ 5430  1569]]
0.94
0.94
0.8847 = roc_auc_score  predict_proba_train
0.8611 = roc_auc_score  predict_proba_test


#### Алгоритм ECLAT не применялся, тк реализан на языке R. подробнее тут:

https://habr.com/ru/company/ods/blog/353502/

В данной работе отражен только общий ход вычислений.
Объем кода и вычислений был намного больше (тут не отражены).
Были спробованы предсказания с различными комбинациями признаков: комбинированные  и тоько новые.
В различных ситуациях модель предсказывала по разному: при избыточности и недостатке признаков метрики падали.
Максимально возможные метрики без ARL были достигнуты ранее.
При определенном подходе удалось улучшить метрики на 0.004-0.005 от исходных, что позволяет подняться на 100 - 200 единиц в рейтинге этого соревнования. (Отличия 500 и 1 места - 0.009) 


# Выводы: